In [1]:
using Pkg
Pkg.activate("C:\\Users\alili\\.julia\\environments\\v1.5\\Project.toml") # change path 
Pkg.instantiate()

 Activating environment at `F:\Projects\ATIS3_2020\Project.toml`


In [2]:
using JuMP, LinearAlgebra, Plots, StatsPlots, CPLEX, Statistics, CSV, DataFrames

In [3]:
#read scenario data
scenarios_hexagons = CSV.read("scenarios_multinomial.csv", DataFrame)
scenarios_prices = CSV.read("priceScenarions3.csv", DataFrame)

SOC = 0.33
hexagons = names(scenarios_hexagons)[1:length(names(scenarios_hexagons))-1] # vector of hexagons (strings)
hours = names(scenarios_prices)[3:length(names(scenarios_prices))] #vector of hours for charging (strings)
car_kWh = 30 # capacity of each car

#demand mxn (m=hexagon, n=scenario) in kWh summe über car*max_kWh*(1-SOC)
demand_cars = transpose(convert(Matrix,scenarios_hexagons[:,hexagons]))
demand_kWh = demand_cars.*car_kWh.*(1-SOC)

#prices mxn (m=hours, n=scenario)
prices = transpose(convert(Matrix,scenarios_prices[:,hours]))

probability_hexagons = transpose(scenarios_hexagons.probs) # probabilities for each scenario (#cars per hexagon)
probability_prices = transpose(scenarios_prices.Probability) # probabilities for each scenario (timeseries prices)

P = [3, 7, 11] # different charger types in kW
charger_cost = [1500, 800, 500] # investment cost for charger 3 kW, 7 kW und 11 kW in €/kW
opportunity_cost = 5000 # cost of demand not served


5000

## Build the scenario tree

In [4]:
## Build Probabilities
comb_probability = []
for prob_d in probability_hexagons
    for prob_p in probability_prices
        push!(comb_probability,prob_d*prob_p)
    end
end

## Build Demand kWh and Demand Cars
num_hex = size(demand_kWh,1)
num_Scen_Dem = size(demand_kWh,2)
num_Scen_Pri = size(prices,2)

comb_demand_kWh = zeros((num_hex, num_Scen_Dem*num_Scen_Pri))
comb_demand_cars = zeros((num_hex, num_Scen_Dem*num_Scen_Pri))
counter = 0

for num_d in 1:num_Scen_Dem
    for num_p in 1:num_Scen_Pri
        counter += 1
        comb_demand_kWh[1:num_hex,counter] = demand_kWh[1:num_hex,num_d]
        comb_demand_cars[1:num_hex,counter] = demand_cars[1:num_hex,num_d]
    end
end

## Build Prices
num_Scen_Dem = size(demand_kWh,2)
num_Scen_Pri = size(prices,2)

comb_prices = zeros((length(hours), num_Scen_Dem*num_Scen_Pri))
counter = 0

for num_d in 1:num_Scen_Dem
    for num_p in 1:num_Scen_Pri
        counter += 1
        comb_prices[1:length(hours),counter] = prices[1:length(hours),num_p]
    end
end

probability = comb_probability
demand_kWh = comb_demand_kWh
demand_cars = comb_demand_cars
prices = comb_prices

7×16 Array{Float64,2}:
 40.0  21.0  34.0  32.0  40.0  21.0  …  34.0  32.0  40.0  21.0  34.0  32.0
 38.0  20.0  31.0  29.0  38.0  20.0     31.0  29.0  38.0  20.0  31.0  29.0
 33.0  17.0  26.0  27.0  33.0  17.0     26.0  27.0  33.0  17.0  26.0  27.0
 33.0   0.0  15.0  27.0  33.0   0.0     15.0  27.0  33.0   0.0  15.0  27.0
 33.0   0.0   9.0  24.0  33.0   0.0      9.0  24.0  33.0   0.0   9.0  24.0
 32.0  -9.0   5.0  23.0  32.0  -9.0  …   5.0  23.0  32.0  -9.0   5.0  23.0
 32.0  -4.0   1.0  22.0  32.0  -4.0      1.0  22.0  32.0  -4.0   1.0  22.0

In [5]:
function charger_siting(SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, prices, 
        probability, P, charger_cost, opportunity_cost)
    
    # Define Sets
    Γ = Array{Int}(1:length(hexagons)) # Set with hexagons
    Σ = Array{Int}(1:length(P)) # Set with charger types 
    Ω = Array{Int}(1:length(probability)) # Set with scenarios
    Τ = Array{Int}(1:length(hours)) # Set with scenarios for prices

    # Initialize model    
    m = Model(CPLEX.Optimizer)

    # Define variables
    @variable(m, 0 <= x_charger[i=Γ,j=Σ],Int) # number of specific charger type in each hexagon
    @variable(m, 0 <= E_supplied[i=Γ,j=Σ,k=Ω,t=Τ]) # supplied energy in each hexagon kWh
    @variable(m, 0 <= E_not_supplied[i=Γ,k=Ω])     # demand not covered kWh
    @variable(m, 0 <= cars_supplied[i=Γ,j=Σ,k=Ω],Int)
    @variable(m, 0 <= cars_not_supplied[i=Γ,j=Σ,k=Ω],Int)

    # Objective function 
    @expression(m, invest_cost[i=Γ,j=Σ], charger_cost[j].*x_charger[i,j].*P[j]) # first stage investment cost
    @expression(m, charging_opportunity_cost[i=Γ,k=Ω], opportunity_cost.*E_not_supplied[i,k].*probability[k])
    @expression(m, grid_usage[i=Γ,k=Ω,t=Τ], 0.1*sum(E_supplied[i,j,k,t] for j in Σ)^2
                                                     .*probability[k])
    @expression(m, charging_cost[i=Γ,j=Σ,k=Ω,t=Τ], prices[t,k].*E_supplied[i,j,k,t]
                                                     .*probability[k]) #minimize cost of charging

    @objective(m, Min, sum(invest_cost) 
                     + sum(charging_opportunity_cost)
                     + sum(grid_usage)
                     + sum(charging_cost)
                      )

    # Constraints
    # Eq. Balance equation: Demand in each hexagon has to equal demand. Loss of load is possible
    @constraint(m, eq_balance[i=Γ,k=Ω], sum(sum(E_supplied[i,j,k,t] for j in Σ) for t in Τ) + E_not_supplied[i,k] == demand_kWh[i,k])

    # Eq. energy limit: rated_power*chargingtime must not exeed energy
    @constraint(m, eq_cap[i=Γ,j=Σ,k=Ω,t=Τ], E_supplied[i,j,k,t] <= x_charger[i,j].*P[j])

    # Eq. supplied cars and not supplied cars have to equal amount of cars in each hexagon
    @constraint(m, eq_cars_supplied[i=Γ,j=Σ,k=Ω], sum(cars_supplied[i,j,k] for j in Σ) 
                                                    + sum(cars_not_supplied[i,j,k] for j in Σ)
                                                   == demand_cars[i,k])

    # Eq. maximum one car per charging station
    @constraint(m, eq_max_cars[i=Γ,j=Σ,k=Ω], cars_supplied[i,j,k] <= x_charger[i,j])

    # Eq. 
    @constraint(m, eq_max_supply[i=Γ,j=Σ,k=Ω], sum(E_supplied[i,j,k,t] for t in Τ) <= cars_supplied[i,j,k].*car_kWh.*(1-SOC))


    # Run optimization
    optimize!(m)

    # Evaluate results
    obj = objective_value(m)
    charger_in_hex = value.(x_charger)
    
    return charger_in_hex, x_charger, E_not_supplied, E_supplied, cars_not_supplied, cars_supplied, obj
end


charger_siting (generic function with 1 method)

In [6]:
charger_in_hex, x_charger, E_not_supplied, E_supplied, cars_not_supplied, cars_supplied, obj= charger_siting(
    SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, prices, 
        probability, P, charger_cost, opportunity_cost)

println(charger_in_hex)
println(obj)

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
Found incumbent of value 5.9192877e+07 after 0.00 sec. (3.98 ticks)
Tried aggregator 1 time.
MIQP Presolve eliminated 18184 rows and 16693 columns.
Reduced MIQP has 29928 rows, 27054 columns, and 101136 nonzeros.
Reduced MIQP has 608 binaries, 3742 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 65016 nonzeros.
Presolve time = 0.06 sec. (67.57 ticks)
Probing time = 0.03 sec. (1.51 ticks)
Tried aggregator 1 time.
MIQP Presolve eliminated 116 rows and 107 columns.
Reduced MIQP has 29812 rows, 26947 columns, and 100744 nonzeros.
Reduced MIQP has 592 binaries, 3739 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 64764 nonzeros.
Presolve time = 0.03 sec. (44.11 ticks)
Classifier predicts products in MIQP should be linearized.
Probing time = 0.02 sec. (1.50 ticks)
Clique table members: 148.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: 